## Creating input-target pairs

In this section we implement a data loader that fetches the input-target pairs using sliding window approach

In [1]:
import importlib
import importlib.metadata
import tiktoken
print("tiktoken version: ", importlib.metadata.version("tiktoken"))

tiktoken version:  0.9.0


In [2]:
tokenizer = tiktoken.get_encoding('gpt2')

In [3]:
with open("the-verdict.txt", 'r', encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


we remove the first 50 tokens form the dataset for demonstration purposed as it results in slightly more interesting

In [4]:
enc_sample = enc_text[50:]

The context size determines the how many tokens are included in the input

In [5]:
context_size = 4 # length of the input
# the context size of 4 means that the model is trained to look at a sequence of 4 word
# to predict the next word in the sentence.
# the input x is the first 4 tokens [1, 2, 3, 4], and the target  is the next 4 token
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y: {y}")

x: [290, 4920, 2241, 287]
y: [4920, 2241, 287, 257]


In [6]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "------->", desired)

[290] -------> 4920
[290, 4920] -------> 2241
[290, 4920, 2241] -------> 287
[290, 4920, 2241, 287] -------> 257


Everything left of the arrow refers to the input and LLM would receive and the token ID on the right, LLM is supposed to predict

In [7]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "------->", tokenizer.decode([desired]))

 and ------->  established
 and established ------->  himself
 and established himself ------->  in
 and established himself in ------->  a


There is only one more task before we can turn the tokens into embeddings: implementing an efficient data loader that interates over the input datasets and returns the inputs and targets as pytorch tensors, which can be thought of  as multidimensional arrays

### Implementing Data Loader

step1: Tokenize the entire text

step2: Use a sliding window to chunk the book into overlapping sequences of max_length

setp3: Return the total number of rows in the dataset

step4: Return a single row from the dataset

In [8]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_lengths
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i+1:i+max_length +1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

The following code will use the GPTDatasetV1 to load the inputs in batches via a pytorch DataLoader

step1: Inintialize the tokenizer

step2: Create dataset

step3: drop_last=True drops the batch if it is shorter than the specified batch_size to prevent loss spikes during training

step4: the number of cpu processes to use for preprocessing

In [9]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

Lets test the dataloader with batch size of 1 for an LLM with context size of 4
This will develop an intuition of how the GPTDatasetV1 class and create_dataloader_v1 function work together:

In [10]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

### Convert dataloader into python iterator to fetch next entry via python's built-in next() function

In [11]:
import torch
print("Pytorch version:", torch.__version__)
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
                                  )
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

Pytorch version: 2.5.1+cu121
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


The first_batch variables contains the two tensors: the first tensor stores the input token IDs and the second tensor stores target token IDs

In [12]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [13]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False
                                  )
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])]


## Understanding and implementing token embeddings

In [1]:
pip install gensim

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------------- ------------------ 30.7/61.0 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 819.4 kB/s eta 0:00:00
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 60.6/60.6 kB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.0 MB 6.4 MB/s eta 0:00:04
   ---------------------------------------- 0.2/24.0 MB 2.1 MB/s eta 0:00:12
    --------------------------------------- 0.4/24.0 MB 2.7 MB/s eta 0:00:09
    --------------------------------------- 0.6/24.0 MB 2.9 MB/s eta 0:00:08
   - -------------------------------------- 0.8/24.0 MB 3.3 MB/s eta 0:00:08
   - -------------------------------------- 1.0/24.0 MB 3.6 MB/s eta 0:00:07
   -- ------------------------------------- 1.2/24.0 MB 3.7 MB/s eta 0:00:07
   -- -

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import trained model

In [2]:
import gensim.downloader as api
model = api.load("word2vec-google-news-300") # download the model and returns as object ready for use

[==================================================] 100.0% 1662.8/1662.8MB downloaded


#### Example if a word as a vector

In [3]:
word_vectors = model
# let us look how the vector embeddings of word looks like
print(word_vectors['computer']) 
# example: accessing the vector for word 'computer'

[ 1.07421875e-01 -2.01171875e-01  1.23046875e-01  2.11914062e-01
 -9.13085938e-02  2.16796875e-01 -1.31835938e-01  8.30078125e-02
  2.02148438e-01  4.78515625e-02  3.66210938e-02 -2.45361328e-02
  2.39257812e-02 -1.60156250e-01 -2.61230469e-02  9.71679688e-02
 -6.34765625e-02  1.84570312e-01  1.70898438e-01 -1.63085938e-01
 -1.09375000e-01  1.49414062e-01 -4.65393066e-04  9.61914062e-02
  1.68945312e-01  2.60925293e-03  8.93554688e-02  6.49414062e-02
  3.56445312e-02 -6.93359375e-02 -1.46484375e-01 -1.21093750e-01
 -2.27539062e-01  2.45361328e-02 -1.24511719e-01 -3.18359375e-01
 -2.20703125e-01  1.30859375e-01  3.66210938e-02 -3.63769531e-02
 -1.13281250e-01  1.95312500e-01  9.76562500e-02  1.26953125e-01
  6.59179688e-02  6.93359375e-02  1.02539062e-02  1.75781250e-01
 -1.68945312e-01  1.21307373e-03 -2.98828125e-01 -1.15234375e-01
  5.66406250e-02 -1.77734375e-01 -2.08984375e-01  1.76757812e-01
  2.38037109e-02 -2.57812500e-01 -4.46777344e-02  1.88476562e-01
  5.51757812e-02  5.02929

In [4]:
print(word_vectors['cat'].shape)

(300,)


### king + woman - man = ?

In [5]:
# Example of using most_similar
print(word_vectors.most_similar(positive=['king', 'woman'], negative=['man']))

[('queen', 0.7118193507194519), ('monarch', 0.6189674139022827), ('princess', 0.5902431011199951), ('crown_prince', 0.5499460697174072), ('prince', 0.5377321839332581), ('kings', 0.5236844420433044), ('Queen_Consort', 0.5235945582389832), ('queens', 0.5181134343147278), ('sultan', 0.5098593831062317), ('monarchy', 0.5087411999702454)]


In [8]:
print(word_vectors.most_similar(positive=['king', 'son'], negative=['daughter'] ))

[('kings', 0.6610661745071411), ('crown_prince', 0.539524257183075), ('prince', 0.5256890058517456), ('ruler', 0.509535551071167), ('monarch', 0.5093126893043518), ('Simeon_Saxcoburggotski', 0.5077420473098755), ('sultan', 0.5074067711830139), ('King', 0.49446699023246765), ('Majesty_King', 0.4904979467391968), ('princes', 0.4890645146369934)]


In [9]:
# example of calculating similarity
print(word_vectors.similarity('woman', 'man'))
print(word_vectors.similarity('king', 'queen'))
print(word_vectors.similarity('paper', 'water'))

0.76640123
0.6510957
0.11408084
